In [ ]:
DATA_PROCESSED_DIR = "../input/mini-project/files/_data_processed/"

In [ ]:
CSV_FILES="../input/csv-files"
CSV_FILES

In [ ]:
#model making
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import seaborn as sn
print (tf.__version__)

### Load Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
IMAGES_DIR = DATA_PROCESSED_DIR
IMAGES_DIR

In [ ]:
CLASS_NAMES = "Atelectasis,Cardiomegaly,Effusion,\
Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,\
Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia"
CLASS_NAMES = CLASS_NAMES.split(',')
CLASS_NAMES

## Data Pipeline

In [ ]:
from imgaug import augmenters as iaa

AUG = iaa.Sequential(
    [
        iaa.Fliplr(0.5),
    ],
    random_order=True,
)

In [ ]:
#data pipeline
class DataGenerator(Sequence):
    """
    This is the Sequece data generator
    """

    def __init__(self, dataset_csv_file, class_names, source_image_dir, batch_size, target_size, verbose, 
                 shuffle_on_epoch_end, counts, augmenter):
        """
        :param dataset_csv_file: str, path of dataset csv file
        :param class_names: list of str
        :param batch_size: int
        :param target_size: tuple(int, int)
        :param verbose: int
        """
        self.dataset_df = pd.read_csv(dataset_csv_file)
        self.source_image_dir = source_image_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.verbose = verbose
        self.augmenter = augmenter
        self.shuffle = shuffle_on_epoch_end
        self.random_state = 1
        self.class_names = class_names
        self.counts = counts
        self.steps = int(np.ceil(self.counts / float(self.batch_size)))
        self.prepare_dataset()
        
        
    def __bool__(self):
        return True

    def __len__(self):
        return self.steps

    def __getitem__(self, idx):
        batch_x_path = self.x_path[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = np.asarray([self.load_image(x_path) for x_path in batch_x_path])
        batch_x = self.transform_batch_images(batch_x)
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

    def load_image(self, image_file):
        image_path = os.path.join(self.source_image_dir, image_file)
        image = Image.open(image_path)
        image_array = np.asarray(image.convert("RGB"))
#         image_array = image_array / 255.
        return image_array

    def transform_batch_images(self, batch_x):
        if self.augmenter is not None:
            batch_x = self.augmenter.augment_images(batch_x)
        return batch_x

    def get_y_true(self):
        """
        Use this function to get y_true for predict_generator
        In order to get correct y, you have to set shuffle_on_epoch_end=False.
        """
        if self.shuffle:
            raise ValueError("""
            You're trying run get_y_true() when generator option 'shuffle_on_epoch_end' is True.""")
    
    def prepare_dataset(self):
        df = self.dataset_df.sample(frac=1., random_state=self.random_state)
        self.x_path, self.y = df["Image Index"].to_numpy(), df[self.class_names].to_numpy()

    def on_epoch_end(self):
        if self.shuffle:
            self.random_state += 1
            self.prepare_dataset()
            

In [ ]:
def get_class_weights(total_counts, class_positive_counts, multiply):
    """
    Calculate class_weight used in training
    Arguments:
    total_counts - int
    class_positive_counts - dict of int, ex: {"Effusion": 300, "Infiltration": 500 ...}
    multiply - int, positve weighting multiply
    use_class_balancing - boolean 
    Returns:
    class_weight - dict of dict, ex: {"Effusion": { 0: 0.01, 1: 0.99 }, ... }
    """
    def get_single_class_weight(pos_counts, total_counts):
        denominator = (total_counts - pos_counts) * multiply + pos_counts
        return {
            0: pos_counts / denominator,
            1: (denominator - pos_counts) / denominator,
        }

    class_names = list(class_positive_counts.keys())
    label_counts = np.array(list(class_positive_counts.values()))
    class_weights = []
    for i, class_name in enumerate(class_names):
        class_weights.append(get_single_class_weight(label_counts[i], total_counts))

    return class_weights

### Class weights generartor
* Due to imbalanced distribution of dataset.

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_names):
    df = pd.read_csv(os.path.join(CSV_FILES, "train.csv"))
    class_series = df[class_names].to_numpy()
    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(f"No of Train Images are {n_samples}")
    
    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) 
    return dict(zip(class_labels, class_weights))

In [ ]:
print (generate_class_weights(CLASS_NAMES))

In [ ]:
def get_sample_weights(total_counts, class_positive_counts, multiply):
    def get_single_class_weight(pos_counts, total_counts):
        denominator = (total_counts - pos_counts) * multiply + pos_counts
        return {
            0: pos_counts / denominator,
            1: (denominator - pos_counts) / denominator,
        }

    class_names = list(class_positive_counts.keys())
    label_counts = np.array(list(class_positive_counts.values()))
    sample_weights = []
    for i, class_name in enumerate(class_names):
        sample_weights.append(get_single_class_weight(label_counts[i], total_counts))

    return sample_weights

In [ ]:
def get_sample_counts(csv_dir, dataset, class_names):
    """
    Get total and class-wise positive sample count of a dataset
    """
    df = pd.read_csv(os.path.join(csv_dir, f"{dataset}.csv"))
    total_count = df.shape[0]
    labels = df[class_names].to_numpy()
    positive_counts = np.sum(labels, axis=0)
    class_positive_counts = dict(zip(class_names, positive_counts))
    return total_count, class_positive_counts

In [ ]:
output_dir= "../input/chexnetkeras/"


In [ ]:
train_counts, train_pos_counts = get_sample_counts(csv_dir=CSV_FILES ,dataset="train", class_names=CLASS_NAMES)
dev_counts, _ = get_sample_counts(csv_dir=CSV_FILES, dataset="dev", class_names=CLASS_NAMES)
test_counts, _ = get_sample_counts(output_dir, "test", CLASS_NAMES)

In [ ]:
print(f'No of Train Images = {train_counts} | No of Validation Images = {dev_counts}')


In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
INI_LE_RATE = 0.001

# NO of CPU for pipeline
GENRATOR_WORKERS = 8
# Image Dimention
IMAGE_DIM = 256

In [ ]:
train_sequence = DataGenerator(
    dataset_csv_file=os.path.join(CSV_FILES, "train.csv"),
    class_names=CLASS_NAMES,
    source_image_dir=IMAGES_DIR,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_DIM, IMAGE_DIM, 3),
    verbose=1,
    shuffle_on_epoch_end=True,
    counts=train_counts,
    augmenter=AUG,
)

In [ ]:
validation_sequence = DataGenerator(
    dataset_csv_file=os.path.join(CSV_FILES, "dev.csv"),
    class_names=CLASS_NAMES,
    source_image_dir=IMAGES_DIR,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_DIM, IMAGE_DIM, 3),
    verbose=1,
    shuffle_on_epoch_end=True,
    counts=dev_counts,
    augmenter=AUG
)

In [ ]:
test_sequence = DataGenerator(
    dataset_csv_file="../input/chexnetkeras/test.csv",
    class_names=CLASS_NAMES,
    source_image_dir=IMAGES_DIR,
    batch_size=BATCH_SIZE,
    target_size=(IMAGE_DIM, IMAGE_DIM, 3),
    verbose=1,
    shuffle_on_epoch_end=False,
    counts=test_counts,
    augmenter=AUG
)

In [ ]:
from PIL import Image
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
plt.imshow(train_sequence[0][0][0])

### Barplot to represent the Train Datset

In [ ]:
df = pd.read_csv(os.path.join(CSV_FILES, "train.csv"))
total_count = df.shape[0]
categories = list(CLASS_NAMES)
sns.set(font_scale = 2)
plt.figure(figsize=(40,12))
ax= sns.barplot(categories, df[CLASS_NAMES].sum().values)
plt.title(" Number of category", fontsize=30)
plt.ylabel('Number of Compilcations', fontsize=20)
plt.xlabel('Type of Complications ', fontsize=20)
#adding the text labels
rects = ax.patches
labels = df[CLASS_NAMES].sum().values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=20)
plt.show()

### Barplot to represent the Multi-labels

In [ ]:
rowSums = df[CLASS_NAMES].sum(axis=1)
multiLabel_counts = rowSums.value_counts()
multiLabel_counts = multiLabel_counts.iloc[1:]
sns.set(font_scale = 2)
plt.figure(figsize=(20,8))
ax = sns.barplot(multiLabel_counts.index, multiLabel_counts.values)
plt.title("Images with multiple labels ")
plt.ylabel('Number of Complications', fontsize=18)
plt.xlabel('Number of labels', fontsize=18)#adding the text labels
rects = ax.patches
labels = multiLabel_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show()

## Training the models

In [ ]:


import importlib
from tensorflow.keras.applications import inception_v3,resnet50,mobilenet_v2,densenet
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow import concat
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.math import reduce_logsumexp
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard
from tensorflow.keras import Model
from tensorflow.keras.metrics import AUC, Accuracy, BinaryAccuracy
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score


#### Important parameters

In [ ]:
init_learning_rate = 1e-4
EPSILON = 1e-4 # AdamOptimizer epsilon
dropout_rate = 0.2

In [ ]:
nesterov_momentum = 0.9
weight_decay = 1e-4

### Y Test true labels

In [ ]:
df = pd.read_csv("../input/chexnetkeras/test.csv")
total_count = df.shape[0]
y_true = df[CLASS_NAMES].to_numpy()

In [ ]:
DEF_MODEL = [["DenseNet121","densenet"],["ResNet50","resnet50"],["InceptionV3","inception_v3"],["MobileNetV2","mobilenet_v2"]]

In [ ]:
def get_model(class_names, model=DEF_MODEL[0], input_shape=None, input_tensor=None, train=False, weights=None):
    base_weights = weights
    base_model_class = getattr(
        importlib.import_module(
            f"tensorflow.keras.applications.{model[1]}"
        ),
        model[0])
    img_input = input_tensor
    base_model = base_model_class(
        include_top=False,
        input_tensor=input_tensor,
        input_shape=input_shape,
        weights=base_weights,
    )
    base_model.trainable = train
    return base_model

#### Defining the Input

In [ ]:
INPUT_SHAPE = (IMAGE_DIM, IMAGE_DIM, 3)
IMG_INPUT = tf.keras.Input(INPUT_SHAPE,name='Input')

In [ ]:
def lse_pool(x):
    return reduce_logsumexp(x,axis=[1,2],keepdims=True,name='lse_pool')

In [ ]:
if not os.path.exists("/kaggle/working/logs"):
    os.mkdir("/kaggle/working/logs")
LOGS_PATHS = "/kaggle/working/logs"



In [ ]:
if not os.path.exists("/kaggle/working/checkpoints"):
    os.mkdir("/kaggle/working/checkpoints")
CHECK_POINT = "/kaggle/working/checkpoints"

if not os.path.exists("/kaggle/working/weights"):
    os.mkdir("/kaggle/working/weights")
WEIGHTS_PATH = "/kaggle/working/weights"

In [ ]:
training_stats = {}
train_steps = int(train_counts / BATCH_SIZE)
validation_steps = int(dev_counts / BATCH_SIZE)
test_steps = int(test_counts / BATCH_SIZE)
POS_WEIGHTS_MUL=1

CONFUSION MATRIX CODE****

In [ ]:
def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )

    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for the class - " + class_label)

### DenseNet Model 

In [ ]:
y_true_argmax=np.argmax(y_true, axis=1)

#### Model creation

In [ ]:
IMAGE_INPUT = tf.keras.applications.densenet.preprocess_input(IMG_INPUT)
DenseNet = get_model(class_names=CLASS_NAMES,
                   model=DEF_MODEL[0],
                   input_shape=INPUT_SHAPE,
                   input_tensor=IMG_INPUT,
                   train=True,weights="imagenet")

In [ ]:
x = DenseNet.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x.get_shape()

In [ ]:
x = Dropout(0.2)(x)
x = Dense(1024,activation='relu')(x) 
x = Dense(512,activation='relu')(x) 
x = Dropout(0.2)(x)
predictions = Dense(len(CLASS_NAMES), activation="sigmoid", name="predictions")(x)
dense_model = Model(inputs=IMG_INPUT, outputs=predictions)

#### Model creation end

### Callback parameters

#### Making directories for storing weights, logs, checkpoints

In [ ]:
DENSE_CHKP = os.path.join(CHECK_POINT,DEF_MODEL[0][0])
DENSE_WEIGHTS = os.path.join(WEIGHTS_PATH, DEF_MODEL[0][0])
DENSE_LOGS = os.path.join(LOGS_PATHS, DEF_MODEL[0][0])

In [ ]:
# DenseNet Chechpoint directory
os.mkdir(DENSE_CHKP)
# DenseNet Weights directory
os.mkdir(DENSE_WEIGHTS)
# DenseNet Logs directory
os.mkdir(DENSE_LOGS)

#### Class weights 

In [ ]:
classWeights = generate_class_weights(CLASS_NAMES)
sampleWeights = get_sample_weights(train_counts, train_pos_counts,multiply=1)
print("** class_weights **")
print(classWeights)

#### Checkpoints 

In [ ]:
checkpoint = ModelCheckpoint(
    filepath=os.path.join(DENSE_CHKP,f"chkp_best.hdf5"),
    save_weights_only=True,
    save_best_only=True,
    mode='max',
    verbose=1,
    save_freq="epoch",
)

#### Defining callbacks for DenseNet

In [ ]:
callbacks = [
    checkpoint,
    TensorBoard(log_dir=DENSE_LOGS,histogram_freq=0, write_graph=True, write_images=True,update_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                      verbose=1, mode="min", min_lr=1e-8),
]

#### Metrics

In [ ]:
METRICS = [BinaryAccuracy(name='accuracy'),AUC(name='AUC',multi_label=True),'hinge']

### Compiling the model

In [ ]:
import keras.backend as K

def loss_pred(y_true, y_pred):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=tf.cast(y_true,tf.float32))

    loss = tf.reduce_mean(tf.reduce_sum(cross_entropy, axis=1))
    return loss


In [ ]:
print("** compile model with class weights **")
initial_learning_rate=0.001
optimizer = Adam(lr=initial_learning_rate)
dense_model.compile(optimizer=optimizer, loss="binary_crossentropy",metrics=METRICS)

### Model Summary

In [ ]:
# dense_model.summary()

In [ ]:
print(f"Total number of parameters is {dense_model.count_params()}")

In [ ]:
hist1 = dense_model.fit(x=train_sequence,
                        batch_size=32,
                        validation_data=validation_sequence,
                        verbose=1,
                        epochs=EPOCHS,
                        workers=4,
                        class_weight=classWeights,
                        callbacks=callbacks)

#### Plot training & validation accuracy values

In [ ]:
plt.plot(hist1.history['AUC'])
plt.plot(hist1.history['val_AUC'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

#### Plot training & validation loss values

In [ ]:
plt.plot(hist1.history['loss'])
plt.plot(hist1.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### Save Weights

In [ ]:
dense_model.save_weights("dense_net_weights.h5")

### Predictions

In [ ]:
y_pred_dense = dense_model.predict(x=test_sequence,
                                   verbose=1,
                                   workers=4)


#### Multi-label confusion matrix

In [ ]:
# confusion_matrix=multilabel_confusion_matrix(y_true, y_pred_dense)
# df_cm = pd.DataFrame(confusion_matrix, index = [i for i in "ABCDEFGHIJKLMO"],
#                   columns = [i for i in "ABCDEFGHIJKLMO"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

y_pred_dense_argmax = np.argmax(y_pred_dense, axis=1)
confusion_matrix=multilabel_confusion_matrix(y_true_argmax, y_pred_dense_argmax)

#### Multi-label classification report

In [ ]:
# print("\nClassification report : \n", classification_report(y_true, y_pred_dense, target_names=CLASS_NAMES))

fig, ax = plt.subplots(7, 2, figsize=(25, 15))
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_matrix, CLASS_NAMES):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

In [ ]:
print("\nClassification report : \n", classification_report(y_true_argmax, y_pred_dense_argmax, target_names=CLASS_NAMES))

## ResNet50 model

In [ ]:
IMAGE_INPUT = tf.keras.applications.resnet.preprocess_input(IMG_INPUT)
ResNet = get_model(class_names=CLASS_NAMES,
                   model=DEF_MODEL[1],
                   input_shape=INPUT_SHAPE,
                   input_tensor=IMG_INPUT,
                   train=True,weights="imagenet")

In [ ]:
x = ResNet.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x.get_shape()

In [ ]:
predictions = Dense(len(CLASS_NAMES), activation="sigmoid", name="predictions")(x)
res_model = Model(inputs=IMG_INPUT, outputs=predictions)

In [ ]:
RES_CHKP = os.path.join(CHECK_POINT,DEF_MODEL[1][0])
RES_WEIGHTS = os.path.join(WEIGHTS_PATH, DEF_MODEL[1][0])
RES_LOGS = os.path.join(LOGS_PATHS, DEF_MODEL[1][0])

In [ ]:
# DenseNet Chechpoint directory
os.mkdir(RES_CHKP)
# DenseNet Weights directory
os.mkdir(RES_WEIGHTS)
# DenseNet Logs directory
os.mkdir(RES_LOGS)

In [ ]:
checkpoint = ModelCheckpoint(
    filepath=os.path.join(RES_CHKP,f'chkp_best.hdf5'),
    save_weights_only=True,
    save_best_only=False,
    verbose=1,
    save_freq='epoch',
)

In [ ]:
callbacks = [
    checkpoint,
    TensorBoard(log_dir=RES_LOGS,histogram_freq=0, write_graph=True, write_images=True,update_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                      verbose=1, mode="min", min_lr=1e-8),
]

In [ ]:
print("** compile model with class weights **")
initial_learning_rate=0.001
optimizer = Adam(lr=initial_learning_rate)
res_model.compile(optimizer=optimizer, loss="binary_crossentropy",metrics=METRICS)

In [ ]:
# model.summary()

In [ ]:
print(f"Total number of parameters is {res_model.count_params()}")

In [ ]:
hist2 = res_model.fit(x=train_sequence,
                      batch_size=32,
                      validation_data=validation_sequence,
                      verbose=1,
                      epochs=EPOCHS,
                      workers=4,
                      class_weight=classWeights,
                      callbacks=callbacks)

#### summarize history for accuracy


In [ ]:
plt.plot(hist2.history['accuracy'])
plt.plot(hist2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### summarize history for loss


In [ ]:
plt.plot(hist2.history['loss'])
plt.plot(hist2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Save Weights

In [ ]:
res_model.save_weights("res_net_weights.h5")

### Predictions

In [ ]:
y_pred_res = res_model.predict(x=test_sequence,
                               verbose=1,
                               workers=4)

#### Multi-label confusion matrix

In [ ]:
# confusion_matrix=multilabel_confusion_matrix(y_true, y_pred_res)
# df_cm = pd.DataFrame(confusion_matrix, index = [i for i in "ABCDEFGHIJK"],
#                   columns = [i for i in "ABCDEFGHIJK"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)
y_pred_res_argmax = np.argmax(y_pred_res, axis=1)
confusion_matrix=multilabel_confusion_matrix(y_true_argmax, y_pred_res_argmax)

fig, ax = plt.subplots(7, 2, figsize=(25, 15))
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_matrix, CLASS_NAMES):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()


#### Multi-label classification report

In [ ]:
print("\nClassification report : \n",classification_report(y_true_argmax, y_pred_res_argmax, target_names=CLASS_NAMES))

## InceptionNet V3

In [ ]:
IMAGE_INPUT = tf.keras.applications.inception_v3.preprocess_input(IMG_INPUT)
IncNet = get_model(class_names=CLASS_NAMES,
                   model=DEF_MODEL[2],
                   input_shape=INPUT_SHAPE,
                   input_tensor=IMG_INPUT,
                   train=True,weights="imagenet")

In [ ]:
x = IncNet.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x.get_shape()

In [ ]:
predictions = Dense(len(CLASS_NAMES), activation="sigmoid", name="predictions")(x)
inc_model = Model(inputs=IMG_INPUT, outputs=predictions)

In [ ]:
INC_CHKP = os.path.join(CHECK_POINT,DEF_MODEL[2][0])
INC_WEIGHTS = os.path.join(WEIGHTS_PATH, DEF_MODEL[2][0])
INC_LOGS = os.path.join(LOGS_PATHS, DEF_MODEL[2][0])

In [ ]:
# DenseNet Chechpoint directory
os.mkdir(INC_CHKP)
# DenseNet Weights directory
os.mkdir(INC_WEIGHTS)
# DenseNet Logs directory
os.mkdir(INC_LOGS)

In [ ]:
checkpoint = ModelCheckpoint(
    filepath=os.path.join(INC_CHKP,f'chkp_best.hdf5'),
    save_weights_only=True,
    save_best_only=True,
    verbose=1,
    save_freq='epoch',
)

In [ ]:
callbacks = [
    checkpoint,
    TensorBoard(log_dir=INC_LOGS,histogram_freq=0, write_graph=True, write_images=True,update_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                      verbose=1, mode="min", min_lr=1e-8),
]

In [ ]:
print("** compile model with class weights **")
initial_learning_rate=0.001
optimizer = Adam(lr=initial_learning_rate)
inc_model.compile(optimizer=optimizer, loss="binary_crossentropy",metrics=METRICS)

In [ ]:
print(f"Total number of parameters is {inc_model.count_params()}")

In [ ]:
hist3 = inc_model.fit(x=train_sequence,batch_size=32,
                      validation_data=validation_sequence,
                      verbose=1,
                      epochs=EPOCHS,
                      workers=4,
                      class_weight=classWeights,
                      callbacks=callbacks)

#### summarize history for accuracy


In [ ]:
plt.plot(hist3.history['accuracy'])
plt.plot(hist3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### summarize history for loss


In [ ]:
plt.plot(hist3.history['loss'])
plt.plot(hist3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


### Save Weights

In [ ]:
inc_model.save_weights("inc_net_weights.h5")

### Predictions

In [ ]:
y_pred_inc = inc_model.predict(x=test_sequence,
                               verbose=1,
                               workers=4)

#### Multi-label confusion matrix

In [ ]:
# confusion_matrix=multilabel_confusion_matrix(y_true, y_pred_inc)
# df_cm = pd.DataFrame(confusion_matrix, index = [i for i in "ABCDEFGHIJK"],
#                   columns = [i for i in "ABCDEFGHIJK"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)

y_pred_inc_argmax = np.argmax(y_pred_inc, axis=1)
confusion_matrix=multilabel_confusion_matrix(y_true_argmax, y_pred_inc_argmax)

fig, ax = plt.subplots(7, 2, figsize=(25, 15))
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_matrix, CLASS_NAMES):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

#### Multi-label classification report

In [ ]:
print("\nClassification report : \n", classification_report(y_true_argmax, y_pred_inc_argmax, target_names=CLASS_NAMES))

## MobileNet V2

In [ ]:
IMAGE_INPUT = tf.keras.applications.mobilenet_v2.preprocess_input(IMG_INPUT)
MobileNet = get_model(class_names=CLASS_NAMES,
                   model=DEF_MODEL[3],
                   input_shape=INPUT_SHAPE,
                   input_tensor=IMG_INPUT,
                   train=True, weights="imagenet")

In [ ]:
x = MobileNet.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x.get_shape()

In [ ]:
predictions = Dense(len(CLASS_NAMES), activation="sigmoid", name="predictions")(x)
mobile_model = Model(inputs=IMG_INPUT, outputs=predictions)

In [ ]:
MOB_CHKP = os.path.join(CHECK_POINT,DEF_MODEL[3][0])
MOB_WEIGHTS = os.path.join(WEIGHTS_PATH, DEF_MODEL[3][0])
MOB_LOGS = os.path.join(LOGS_PATHS, DEF_MODEL[3][0])

In [ ]:
# DenseNet Chechpoint directory
os.mkdir(MOB_CHKP)
# DenseNet Weights directory
os.mkdir(MOB_WEIGHTS)
# DenseNet Logs directory
os.mkdir(MOB_LOGS)

In [ ]:
checkpoint = ModelCheckpoint(
    filepath=os.path.join(MOB_CHKP,f'chkp_besT.hdf5'),
    save_weights_only=True,
    save_best_only=False,
    verbose=1,
    save_freq='epoch',
)

In [ ]:
callbacks = [
    checkpoint,
    TensorBoard(log_dir=MOB_LOGS,histogram_freq=0, write_graph=True, write_images=True,update_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                      verbose=1, mode="min", min_lr=1e-8),
]

In [ ]:
print("** compile model with class weights **")
initial_learning_rate=0.001
optimizer = Adam(lr=initial_learning_rate)
mobile_model.compile(optimizer=optimizer, loss="binary_crossentropy",metrics=METRICS)

In [ ]:
print(f"Total number of parameters is {mobile_model.count_params()}")

In [ ]:
hist4 = mobile_model.fit(x=train_sequence,batch_size=32,
                         validation_data=validation_sequence,
                         verbose=1,
                         epochs=EPOCHS,
                         workers=4,
                         class_weight=classWeights,
                         callbacks=callbacks)

In [ ]:
# summarize history for accuracy
plt.plot(hist4.history['accuracy'])
plt.plot(hist4.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(hist4.history['loss'])
plt.plot(hist4.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
mobile_model.save_weights("mobile_net_weights.h5")

### Predictions

In [ ]:
y_pred_mobile = mobile_model.predict(x=test_sequence,
                                     verbose=1,
                                     workers=4)

#### Multi-label confusion matrix

In [ ]:
# confusion_matrix=multilabel_confusion_matrix(y_true, y_pred_mobile)
# df_cm = pd.DataFrame(confusion_matrix, index = [i for i in "ABCDEFGHIJK"],
#                   columns = [i for i in "ABCDEFGHIJK"])
# plt.figure(figsize = (10,7))
# sn.heatmap(df_cm, annot=True)
y_pred_mobile_argmax = np.argmax(y_pred_mobile, axis=1)
confusion_matrix=multilabel_confusion_matrix(y_true_argmax, y_pred_mobile_argmax)
fig, ax = plt.subplots(7, 2, figsize=(25, 15))
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_matrix, CLASS_NAMES):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

#### Multi-label classification report

In [ ]:
print("\nClassification report : \n", classification_report(y_true_argmax, y_pred_mobile_argmax, target_names=CLASS_NAMES))

## Model Factory

In [ ]:
class ModelFactory:
    """
    Model factory for Keras default models
    Select one of: VGG16, VGG19, DenseNet121, ResNet50, InceptionV3, InceptionResNetV2,
    NASNetMobile, NASNetLarge
    """

    def __init__(self):
        self.models_ = dict(
            VGG16=dict(
                input_shape=(256, 256, 3),
                module_name="vgg16",
                last_conv_layer="block5_conv3",
            ),
            VGG19=dict(
                input_shape=(256, 256, 3),
                module_name="vgg19",
                last_conv_layer="block5_conv4",
            ),
            DenseNet121=dict(
                input_shape=(256, 256, 3),
                module_name="densenet",
                last_conv_layer="bn",
            ),
            ResNet50=dict(
                input_shape=(256, 256, 3),
                module_name="resnet50",
                last_conv_layer="activation_49",
            ),
            InceptionV3=dict(
                input_shape=(256, 256, 3),
                module_name="inception_v3",
                last_conv_layer="mixed10",
            ),
            InceptionResNetV2=dict(
                input_shape=(256, 256, 3),
                module_name="inception_resnet_v2",
                last_conv_layer="conv_7b_ac",
            ),
            NASNetMobile=dict(
                input_shape=(256, 256, 3),
                module_name="nasnet",
                last_conv_layer="activation_188",
            ),
            NASNetLarge=dict(
                input_shape=(256, 256, 3),
                module_name="nasnet",
                last_conv_layer="activation_260",
            ),
            MobileNetV2=dict(
                input_shape=(256, 256, 3),
                module_name="mobilenet_v2",
                last_conv_layer="block_16_project_BN"
            ),
        )

    def get_model(self, class_names, model_name="DenseNet121",
                  weights_path=None, input_shape=None, input_tensor=None, train=False):
        
        base_weights = "imagenet"
        base_model_class = getattr(
            importlib.import_module(
                f"tensorflow.keras.applications.{self.models_[model_name]['module_name']}"
            ),
            model_name)
        if input_shape is None:
            input_shape = self.models_[model_name]["input_shape"]
            
        img_input = input_tensor
        base_model = base_model_class(
            include_top=False,
            input_tensor=input_tensor,
            input_shape=input_shape,
            weights=base_weights,
        )
        base_model.trainable = train
        x = base_model(img_input)
        if weights_path == "":
            weights_path = None

        if weights_path is not None:
            print(f"load model weights_path: {weights_path}")
            x.load_weights(weights_path)
        return x
    def Concatenation(self,layers) :
        return tf.concat(layers, axis=3,name='Concatenation')
    
    def transition_layer(self, x):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        in_channel = x.shape[-1]
        x = Conv2D(filters=in_channel*0.5, kernel_size=[1,1])(x)
        return x
    
    def lse_pool(self, cat):
        return reduce_logsumexp(cat,axis=[1,2],keepdims=True)
    
    def build(self, class_names, weights_path=None, inp_shape=None):
        img_input = Input(shape=inp_shape)
        mob_input = mobilenet_v2.preprocess_input(img_input)
        res_input = resnet50.preprocess_input(img_input)
        dense_input= densenet.preprocess_input(img_input)
        inc_input = inception_v3.preprocess_input(img_input)
        input_shape = inp_shape
        x1 = self.get_model(class_names,
                            model_name="ResNet50",
                            weights_path=None,
                            input_shape=input_shape,
                            input_tensor=res_input,
                            train=False)
        x2 = self.get_model(class_names,
                            model_name="InceptionV3",
                            weights_path=None,
                            input_shape=input_shape,
                            input_tensor=inc_input,
                            train=False)
        x3 = self.get_model(class_names,
                            model_name="DenseNet121",
                            weights_path=None,
                            input_tensor=dense_input,
                            train=False)
        x4 = self.get_model(class_names,
                            model_name="MobileNetV2",
                            weights_path=None,
                            input_shape=input_shape,
                            input_tensor=mob_input,
                            train=False)
        
        x2 = ZeroPadding2D(((1,1),(1,1)))(x2)
        x = self.Concatenation([x1,x2,x3,x4])
        print(x.shape[-1])
        x = self.transition_layer(x)
        x = self.lse_pool(x)
        x = Flatten()(x)
        x= Dropout(0.2)(x)
        x= Dense(1024,activation='relu')(x) 
        x= Dense(512,activation='relu')(x) 
        x= Dropout(0.2)(x)
        predictions = Dense(len(class_names), activation="sigmoid", name="predictions")(x)
        return Model(inputs=img_input, outputs=predictions)

#### Defining the model

In [ ]:
model_factory = ModelFactory()
model = model_factory.build(
    CLASS_NAMES,
    weights_path=None,
    inp_shape=(IMAGE_DIM, IMAGE_DIM, 3))

#### Callabcks for the model

#### Chechkpoint

In [ ]:
checkpoint = ModelCheckpoint(
    filepath=os.path.join(WEIGHTS_PATH, "chkp_best_weights.hdf5"),
    save_weights_only=True,
    save_best_only=False,
    verbose=1,
    save_freq='epoch',
)

### Callback

In [ ]:
callbacks = [
    checkpoint,
    TensorBoard(log_dir=LOGS_PATHS,histogram_freq=0, write_graph=True, write_images=True,update_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1,
                      verbose=1, mode="min", min_lr=1e-8),
]

### Metrics

In [ ]:
METRICS = [Accuracy(name='accuracy'),AUC(name='AUC'), BinaryAccuracy('BinAcc')]

#### Now Compiling the model

In [ ]:
model.compile(optimizer='adam', loss=BinaryCrossentropy(from_logits=True),metrics=METRICS)

#### Fitting the model

In [ ]:
hist = model.fit(x=train_sequence,
                 batch_size=32,
                 validation_data=validation_sequence,
                 validation_steps=validation_steps,
                 verbose=1,
                 epochs=10,
                 workers=4)

#### summarize history for accuracy


In [ ]:
plt.plot(hist.history['BinAcc'])
plt.plot(hist.history['val_BinAcc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

y_pred_inc = inc_model.predict(x=test_sequence,
                               verbose=1,
                               workers=4)#### summarize history for loss


In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Predictions

In [ ]:
y_pred = model.predict(x=test_sequence,
                       verbose=1,
                       workers=4)

#### Multi-label confusion matrix

In [ ]:
y_pred_argmax = np.argmax(y_pred, axis=1)
confusion_matrix=multilabel_confusion_matrix(y_true_argmax, y_pred_argmax)
fig, ax = plt.subplots(7, 2, figsize=(25, 15))
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_matrix, CLASS_NAMES):
    print_confusion_matrix(cfs_matrix, axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

#### Multi-label classification report

In [ ]:
print("\nClassification report : \n", classification_report(y_true_argmax, y_pred_argmax,target_names=CLASS_NAMES))